In [1]:
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')

_____

# Process Data methods

In [2]:
def processDataLSA(pathTXT):

    acceptance_pb = []
    acc_best_sol = []
    temp = []
    best_y = []
    curr_y = []
    runtime = 0
    iteration_nbr = 0


    with open(pathTXT, "r") as file:
        dimension = file.readline().count('BEST x')
        best_xs = np.zeros((dimension, 0))
        curr_xs = np.zeros((dimension, 0))

        for line in file:
            if(line.startswith("Runtime")):
                runtime = float(re.sub("[^0-9^.]", "", line))
            elif(line.startswith("Nbr")):
                iteration_nbr = int(re.sub("[^0-9]", "", line))
            else:
                final = 5
                a = line.split("|")
                a = [i.strip() for i in a][:-1]
                acceptance_pb.append(float(a[0]))
                acc_best_sol.append(a[1])
                temp.append(float(a[2]))
                best_y.append(float(a[3]))
                curr_y.append(float(a[4]))

                best_xs_temp = np.zeros((dimension, 1))
                curr_xs_temp = np.zeros((dimension, 1))
                for i in range(dimension):
                    index = i*2
                    best_xs_temp[i] = float(a[final+index])
                    curr_xs_temp[i] = float(a[final+index+1])
                best_xs = np.concatenate((best_xs, best_xs_temp), axis = 1)
                curr_xs = np.concatenate((curr_xs, curr_xs_temp), axis = 1)
                
    acceptance_pb = np.asarray(acceptance_pb)
    acc_best_sol = np.asarray(acc_best_sol)
    temp = np.asarray(temp)
    best_y = np.asarray(best_y)
    curr_y = np.asarray(curr_y)
    
    return acceptance_pb, acc_best_sol, temp, best_y, curr_y, best_xs, curr_xs, runtime, iteration_nbr

In [3]:
def processMarkovLen(init):
    temp = []
    for i, v in enumerate(init):
        if(i>1):
            if(v==1):
                temp.append(init[i-1])
    temp.append(v)
    
    final = list(np.ones(int(temp[0]+1))*temp[0])
    for i in range(len(temp)-1):
        final += list(np.linspace(int(temp[i]), int(temp[i+1]), int(temp[i+1]+1)))[1:]
    return np.asarray(final)

In [4]:
def processDataDSA(pathTXT):
    
    acceptance_pb = []
    acc_best_sol = []
    temp = []
    density = []
    markov_len = []
    best_y = []
    curr_y = []
    runtime = 0
    iteration_nbr = 0
    
    with open(pathTXT, "r") as file:
        dimension = file.readline().count('BEST x')
        best_xs = np.zeros((dimension, 0))
        curr_xs = np.zeros((dimension, 0))

        for line in file:
            if(line.startswith("Runtime")):
                runtime = float(re.sub("[^0-9^.]", "", line))
            elif(line.startswith("Nbr")):
                iteration_nbr = int(re.sub("[^0-9]", "", line))
            else:
                final = 7
                a = line.split("|")
                a = [i.strip() for i in a][:-1]
                acceptance_pb.append(float(a[0]))
                acc_best_sol.append(a[1])
                temp.append(float(a[2]))
                density.append(float(a[3]))
                markov_len.append(float(a[4]))
                best_y.append(float(a[5]))
                curr_y.append(float(a[6]))

                best_xs_temp = np.zeros((dimension, 1))
                curr_xs_temp = np.zeros((dimension, 1))
                for i in range(dimension):
                    index = i*2
                    best_xs_temp[i] = float(a[final+index])
                    curr_xs_temp[i] = float(a[final+index+1])
                best_xs = np.concatenate((best_xs, best_xs_temp), axis = 1)
                curr_xs = np.concatenate((curr_xs, curr_xs_temp), axis = 1)
                
    acceptance_pb = np.asarray(acceptance_pb)
    acc_best_sol = np.asarray(acc_best_sol)
    temp = np.asarray(temp)
    best_y = np.asarray(best_y)
    curr_y = np.asarray(curr_y)
    
    density = np.asarray(density)
    markov_len = processMarkovLen(markov_len)[:len(temp)]
    
    
    return acceptance_pb, acc_best_sol, temp, density, markov_len, best_y, curr_y, best_xs, curr_xs, runtime, iteration_nbr

In [5]:
#path = "../java/SimulatedAnnealing/GraphData/DSA_MinFunction.txt"
#acceptance_pb, acc_best_sol, temp, density, markov_len, best_y, curr_y, best_xs, curr_xs, runtime, iteration_nbr = processDataDSA(path)
#print(len(markov_len), len(temp))

In [6]:
import pandas as pd

def processData4D_panda(pathTXT):
    a = []
    tab = []
    i = 0
    with open(pathTXT, "r") as file:
        for line in file:
            if(line.startswith("Runtime")):
                runtime = float(re.sub("[^0-9^.]", "", line))
            elif(line.startswith("Nbr")):
                iteration_nbr = int(re.sub("[^0-9]", "", line))
            else:
                a = line.split("|")
                a = [i.strip() for i in a][:-1]
                if(i!=0):
                    a = [float(i) for i in a]
                tab.append(a)
            i+=1
            
    titles = tab[0]    
    col_to_del = titles[:5]
    dtf = pd.DataFrame(data=tab[1:],columns=titles)
    afterdrop_all = dtf.drop(col_to_del, axis = 1)

    col_to_del = titles[:5]+titles[5::2]
    afterdrop_cur = dtf.drop(col_to_del, axis = 1)
    
    col_to_del = titles[:5]+titles[6::2]
    afterdrop_bes = dtf.drop(col_to_del, axis = 1)
    
    isCG = dtf['Control-G?']==1.0
    col_to_del = titles[:5]+titles[5::2]
    afterdrop_curCG = dtf[isCG].drop(col_to_del, axis = 1)
    
    return afterdrop_all, afterdrop_cur, afterdrop_bes, afterdrop_curCG

In [7]:
#data, data_cur, data_best, data_curCG = processData4D_panda('../java/SimulatedAnnealing/GraphData/DSA_MinFunction4D.txt')
#data_curCG.head()

In [8]:
import pandas as pd

def processData_stocha(pathTXT):
    a = []
    tab = []
    i = 0
    with open(pathTXT, "r") as file:
        for line in file:
            a = line.split("|")
            a = [i.strip() for i in a][:-1]
            if(i!=0):
                a = [float(i) for i in a]
            tab.append(a)
            i+=1
            
    titles = tab[0]    
    dtf = pd.DataFrame(data=tab[1:],columns=titles)
    
    col_to_del = titles[:5]+titles[6::2]
    afterdrop_bes = dtf.drop(col_to_del, axis = 1)
    
    return afterdrop_bes

In [9]:
data_mult = processData_stocha('../java/SimulatedAnnealing/GraphData/DSA_MinFunction4D_Stocha.txt')
data_mult

,BEST y,BEST x,BEST x2,BEST x3
0,-1.499762,-3.913895,2.359670,2.363568
1,-1.499953,2.351619,2.358217,2.351496
2,-1.499895,-0.780470,2.363617,-3.932070
3,-1.499876,-0.776196,-3.927989,-3.920820
4,-1.499873,-3.936309,2.351115,-3.923217
5,-1.499915,-3.922767,-3.922801,-3.934041
6,-1.499856,-3.924173,-0.786639,2.367808
7,-1.499883,2.346661,-3.931531,-0.783066
8,-1.499899,-3.925888,2.346409,-3.928949
9,-1.499724,-3.934767,-3.913106,2.351392
